In [1]:
#Importing Required Libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf


In [2]:
#Loading the dataset
df=pd.read_csv("/content/breast_cancer.csv")

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 569 entries, 0 to 568
Data columns (total 33 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   id                       569 non-null    int64  
 1   diagnosis                569 non-null    object 
 2   radius_mean              569 non-null    float64
 3   texture_mean             569 non-null    float64
 4   perimeter_mean           569 non-null    float64
 5   area_mean                569 non-null    float64
 6   smoothness_mean          569 non-null    float64
 7   compactness_mean         569 non-null    float64
 8   concavity_mean           569 non-null    float64
 9   concave points_mean      569 non-null    float64
 10  symmetry_mean            569 non-null    float64
 11  fractal_dimension_mean   569 non-null    float64
 12  radius_se                569 non-null    float64
 13  texture_se               569 non-null    float64
 14  perimeter_se             5

In [4]:
df.head()

,id,diagnosis,radius_mean,texture_mean,perimeter_mean,area_mean,smoothness_mean,compactness_mean,concavity_mean,concave points_mean,...,texture_worst,perimeter_worst,area_worst,smoothness_worst,compactness_worst,concavity_worst,concave points_worst,symmetry_worst,fractal_dimension_worst,Unnamed: 32
0,842302,M,17.99,10.38,122.80,1001.0,0.11840,0.27760,0.3001,0.14710,...,17.33,184.60,2019.0,0.1622,0.6656,0.7119,0.2654,0.4601,0.11890,NaN
1,842517,M,20.57,17.77,132.90,1326.0,0.08474,0.07864,0.0869,0.07017,...,23.41,158.80,1956.0,0.1238,0.1866,0.2416,0.1860,0.2750,0.08902,NaN
2,84300903,M,19.69,21.25,130.00,1203.0,0.10960,0.15990,0.1974,0.12790,...,25.53,152.50,1709.0,0.1444,0.4245,0.4504,0.2430,0.3613,0.08758,NaN
3,84348301,M,11.42,20.38,77.58,386.1,0.14250,0.28390,0.2414,0.10520,...,26.50,98.87,567.7,0.2098,0.8663,0.6869,0.2575,0.6638,0.17300,NaN
4,84358402,M,20.29,14.34,135.10,1297.0,0.10030,0.13280,0.1980,0.10430,...,16.67,152.20,1575.0,0.1374,0.2050,0.4000,0.1625,0.2364,0.07678,NaN


In [5]:
#Removing unncecessary features
df=df.drop(columns=['id', 'Unnamed: 32'])

In [6]:
#Checking missing values
df.isnull().sum()

,0
diagnosis,0
radius_mean,0
texture_mean,0
perimeter_mean,0
area_mean,0
smoothness_mean,0
compactness_mean,0
concavity_mean,0
concave points_mean,0
symmetry_mean,0


In [7]:
df.diagnosis.unique()

array(['M', 'B'], dtype=object)

In [8]:
tn={'M':1, 'B':0 }

In [9]:
#Converting target variable to discrete numerical
df['diagnosis'] = df['diagnosis'].map(tn)
df['diagnosis']=df['diagnosis'].astype('float64')

In [10]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 569 entries, 0 to 568
Data columns (total 31 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   diagnosis                569 non-null    float64
 1   radius_mean              569 non-null    float64
 2   texture_mean             569 non-null    float64
 3   perimeter_mean           569 non-null    float64
 4   area_mean                569 non-null    float64
 5   smoothness_mean          569 non-null    float64
 6   compactness_mean         569 non-null    float64
 7   concavity_mean           569 non-null    float64
 8   concave points_mean      569 non-null    float64
 9   symmetry_mean            569 non-null    float64
 10  fractal_dimension_mean   569 non-null    float64
 11  radius_se                569 non-null    float64
 12  texture_se               569 non-null    float64
 13  perimeter_se             569 non-null    float64
 14  area_se                  5

In [11]:
threshold = 0.5

correlation_matrix = df.corr()
high_corr_features = correlation_matrix.index[abs(correlation_matrix["diagnosis"]) > threshold].tolist()
high_corr_features.remove("diagnosis")
print(high_corr_features)



['radius_mean', 'perimeter_mean', 'area_mean', 'compactness_mean', 'concavity_mean', 'concave points_mean', 'radius_se', 'perimeter_se', 'area_se', 'radius_worst', 'perimeter_worst', 'area_worst', 'compactness_worst', 'concavity_worst', 'concave points_worst']


In [12]:
#Selecting predictors and target variables

X_selected = df[high_corr_features]
y = df["diagnosis"]

In [13]:
#Scaling
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

scaler=StandardScaler()
scaled=scaler.fit_transform(X_selected)
sc_t=pd.DataFrame(scaled, columns=X_selected.columns)

In [14]:
#Splitting the data
X_train, X_test, y_train, y_test = train_test_split(X_selected, y, test_size=0.15, random_state=23)

print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

(483, 15)
(86, 15)
(483,)
(86,)


In [15]:
#Importing model and layers
from keras.models import Sequential
from keras.layers import Dense, BatchNormalization

In [16]:
#Model initialization
model=Sequential()

In [17]:
#Adding layer
model.add(Dense(units=15, activation='relu', input_dim=15))
model.add(Dense(units=13, activation='relu'))
model.add(Dense(units=10, activation='relu'))


/usr/local/lib/python3.12/dist-packages/keras/src/layers/core/dense.py:93: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [18]:
#output layer
model.add(Dense(units=1, activation='sigmoid'))

In [19]:
#Compilation
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

In [20]:
#Traing the model
history=model.fit(X_train, y_train, epochs=10, validation_split=0.15)

Epoch 1/10
13/13 ━━━━━━━━━━━━━━━━━━━━ 2s 22ms/step - accuracy: 0.8249 - loss: 0.6027 - val_accuracy: 0.9178 - val_loss: 0.2346
Epoch 2/10
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.8523 - loss: 0.4820 - val_accuracy: 0.9041 - val_loss: 0.2505
Epoch 3/10
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.8600 - loss: 0.3788 - val_accuracy: 0.9041 - val_loss: 0.2226
Epoch 4/10
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.8912 - loss: 0.2939 - val_accuracy: 0.9041 - val_loss: 0.2234
Epoch 5/10
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.9107 - loss: 0.2947 - val_accuracy: 0.9041 - val_loss: 0.2507
Epoch 6/10
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.9027 - loss: 0.2581 - val_accuracy: 0.9041 - val_loss: 0.2227
Epoch 7/10
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.8982 - loss: 0.2610 - val_accuracy: 0.9041 - val_loss: 0.2523
Epoch 8/10
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.8806 - loss: 0.2788 - val_accuracy: 0.8767 - val_loss

In [21]:
model.evaluate(X_test, y_test)

3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - accuracy: 0.9377 - loss: 0.1609


[0.13050664961338043, 0.9534883499145508]

In [22]:
#Hyperparameter tuing using Optuna
!pip install optuna
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.optimizers import Adam

import optuna

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 404.7/404.7 kB 8.5 MB/s eta 0:00:00


In [23]:
def optune(check):
  model=Sequential()
  n_layers=check.suggest_int('n_layers', 1, 3)
  for i in range(n_layers):
    units=check.suggest_int(f'units_{i}', 16, 128)
    model.add(Dense(units, activation='relu'))
    dropout_rate=check.suggest_float(f'dropout_{i}', 0.2, 0.5)
    model.add(Dropout(dropout_rate))

  #ouputlayer 2 layers
  model.add(Dense(1, activation='sigmoid'))

  lr = check.suggest_float('lr', 1e-4, 1e-2, log=True)

  model.compile(
      optimizer=Adam(learning_rate=lr),
      loss='binary_crossentropy',
      metrics=['accuracy']
  )


  history=model.fit(
      X_train, y_train,
      validation_split=0.15,
      epochs=15,
      batch_size=check.suggest_categorical('batch_size', [16, 32, 64]),
      verbose=1
  )

  return max(history.history['val_accuracy'])

In [24]:
#Creating optuna study
study = optuna.create_study(direction='maximize')
study.optimize(optune, n_trials=50)

[I 2025-12-18 06:16:24,918] A new study created in memory with name: no-name-f15ef99e-f51c-4c44-a4c5-eb7760204c68


Epoch 1/15
7/7 ━━━━━━━━━━━━━━━━━━━━ 6s 110ms/step - accuracy: 0.5471 - loss: 44.6624 - val_accuracy: 0.3562 - val_loss: 8.0908
Epoch 2/15
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step - accuracy: 0.4502 - loss: 26.7320 - val_accuracy: 0.3836 - val_loss: 9.1609
Epoch 3/15
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step - accuracy: 0.5182 - loss: 22.6640 - val_accuracy: 0.4932 - val_loss: 1.8629
Epoch 4/15
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step - accuracy: 0.4726 - loss: 22.8654 - val_accuracy: 0.6301 - val_loss: 1.7775
Epoch 5/15
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step - accuracy: 0.5037 - loss: 20.2752 - val_accuracy: 0.4795 - val_loss: 0.9546
Epoch 6/15
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step - accuracy: 0.4890 - loss: 16.0173 - val_accuracy: 0.9041 - val_loss: 0.3119
Epoch 7/15
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step - accuracy: 0.5112 - loss: 13.6986 - val_accuracy: 0.8493 - val_loss: 0.3577
Epoch 8/15
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step - accuracy: 0.5098 - loss: 10.5445 - val_accuracy: 0.6712 - val_loss

[I 2025-12-18 06:16:34,872] Trial 0 finished with value: 0.9041095972061157 and parameters: {'n_layers': 3, 'units_0': 100, 'dropout_0': 0.2992260423163657, 'units_1': 55, 'dropout_1': 0.3072563563447472, 'units_2': 88, 'dropout_2': 0.2969726185841175, 'lr': 0.0007335818511076749, 'batch_size': 64}. Best is trial 0 with value: 0.9041095972061157.


Epoch 1/15
7/7 ━━━━━━━━━━━━━━━━━━━━ 6s 135ms/step - accuracy: 0.5217 - loss: 33.1735 - val_accuracy: 0.3836 - val_loss: 4.8079
Epoch 2/15
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 43ms/step - accuracy: 0.4750 - loss: 26.6421 - val_accuracy: 0.3836 - val_loss: 13.7239
Epoch 3/15
7/7 ━━━━━━━━━━━━━━━━━━━━ 1s 48ms/step - accuracy: 0.4919 - loss: 25.5175 - val_accuracy: 0.4110 - val_loss: 3.3378
Epoch 4/15
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step - accuracy: 0.4889 - loss: 26.9821 - val_accuracy: 0.8356 - val_loss: 1.5438
Epoch 5/15
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - accuracy: 0.4924 - loss: 21.5449 - val_accuracy: 0.6712 - val_loss: 1.2511
Epoch 6/15
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - accuracy: 0.5396 - loss: 19.5388 - val_accuracy: 0.7260 - val_loss: 1.0996
Epoch 7/15
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - accuracy: 0.5580 - loss: 20.6712 - val_accuracy: 0.8904 - val_loss: 1.0055
Epoch 8/15
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - accuracy: 0.5982 - loss: 12.7892 - val_accuracy: 0.8630 - val_los

[I 2025-12-18 06:16:43,033] Trial 1 finished with value: 0.9041095972061157 and parameters: {'n_layers': 2, 'units_0': 97, 'dropout_0': 0.24356015220837052, 'units_1': 124, 'dropout_1': 0.3771387948871684, 'lr': 0.00042703020233322684, 'batch_size': 64}. Best is trial 0 with value: 0.9041095972061157.


...

[I 2025-12-18 06:19:38,376] Trial 47 finished with value: 0.931506872177124 and parameters: {'n_layers': 1, 'units_0': 41, 'dropout_0': 0.4195498427533388, 'lr': 0.00694738608568946, 'batch_size': 16}. Best is trial 11 with value: 0.9452054500579834.


Epoch 1/15
26/26 ━━━━━━━━━━━━━━━━━━━━ 1s 10ms/step - accuracy: 0.5021 - loss: 56.2143 - val_accuracy: 0.3836 - val_loss: 14.4108
Epoch 2/15
26/26 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.5642 - loss: 49.4925 - val_accuracy: 0.3836 - val_loss: 10.0015
Epoch 3/15
26/26 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.5365 - loss: 48.6883 - val_accuracy: 0.4110 - val_loss: 4.6015
Epoch 4/15
26/26 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.5473 - loss: 61.6966 - val_accuracy: 0.5890 - val_loss: 1.6952
Epoch 5/15
26/26 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.5407 - loss: 57.2136 - val_accuracy: 0.4932 - val_loss: 3.2874
Epoch 6/15
26/26 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.5448 - loss: 47.7653 - val_accuracy: 0.4384 - val_loss: 4.0578
Epoch 7/15
26/26 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.4876 - loss: 53.3357 - val_accuracy: 0.5342 - val_loss: 2.7948
Epoch 8/15
26/26 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.5112 - loss: 57.6787 - val_accuracy: 0.4247 

[I 2025-12-18 06:19:41,646] Trial 48 finished with value: 0.7123287916183472 and parameters: {'n_layers': 1, 'units_0': 62, 'dropout_0': 0.43794277547321986, 'lr': 0.00013389522128648467, 'batch_size': 16}. Best is trial 11 with value: 0.9452054500579834.


Epoch 1/15
26/26 ━━━━━━━━━━━━━━━━━━━━ 1s 10ms/step - accuracy: 0.4164 - loss: 61.1471 - val_accuracy: 0.2329 - val_loss: 10.2133
Epoch 2/15
26/26 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.4714 - loss: 54.4221 - val_accuracy: 0.3836 - val_loss: 13.2528
Epoch 3/15
26/26 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.4336 - loss: 45.2038 - val_accuracy: 0.3836 - val_loss: 10.4191
Epoch 4/15
26/26 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.4771 - loss: 38.0762 - val_accuracy: 0.6301 - val_loss: 0.9558
Epoch 5/15
26/26 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.5566 - loss: 27.6503 - val_accuracy: 0.4932 - val_loss: 2.9505
Epoch 6/15
26/26 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.4717 - loss: 31.8696 - val_accuracy: 0.5068 - val_loss: 3.5981
Epoch 7/15
26/26 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.5481 - loss: 23.2066 - val_accuracy: 0.8904 - val_loss: 0.5155
Epoch 8/15
26/26 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.5898 - loss: 23.5773 - val_accuracy: 0.6849

[I 2025-12-18 06:19:45,420] Trial 49 finished with value: 0.9178082346916199 and parameters: {'n_layers': 1, 'units_0': 84, 'dropout_0': 0.39585298126295687, 'lr': 0.0005687952451806742, 'batch_size': 16}. Best is trial 11 with value: 0.9452054500579834.


In [25]:
#Best Hyperparameters found

print('Best Accuracy:', study.best_value)
print('Best Parameters:', study.best_params)

Best Accuracy: 0.9452054500579834
Best Parameters: {'n_layers': 1, 'units_0': 72, 'dropout_0': 0.20460230817107947, 'lr': 0.00012683410854870687, 'batch_size': 16}


In [26]:
#Training final using best params

best_params=study.best_params

model= Sequential()
for i in range(best_params['n_layers']):
  model.add(Dense(best_params[f'units_{i}'], activation='relu'))
  model.add(Dropout(best_params[f'dropout_{i}']))

  model.add(Dense(1, activation='sigmoid'))

  model.compile(
      optimizer=Adam(learning_rate=best_params['lr']),
      loss='binary_crossentropy',
      metrics=['accuracy']

  )

  model.fit(X_train, y_train, epochs=40, batch_size=best_params['batch_size'], verbose=1)

Epoch 1/40
31/31 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.4692 - loss: 31.4640
Epoch 2/40
31/31 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.4990 - loss: 29.9553 
Epoch 3/40
31/31 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.4907 - loss: 31.2094
Epoch 4/40
31/31 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.4586 - loss: 28.0716
Epoch 5/40
31/31 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.5230 - loss: 24.2414
Epoch 6/40
31/31 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.4999 - loss: 25.2237
Epoch 7/40
31/31 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.5211 - loss: 20.7694
Epoch 8/40
31/31 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.5284 - loss: 27.6631
Epoch 9/40
31/31 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.5959 - loss: 21.6144
Epoch 10/40
31/31 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.5572 - loss: 20.9683
Epoch 11/40
31/31 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.5964 - loss: 19.4496
Epoch 12/40
31/31 ━━━━━━━━━━━━━━━━━━━━ 1s 17ms/step - accuracy

In [27]:
#Final model
from sklearn.metrics import accuracy_score, confusion_matrix
y_probs = model.predict(X_test)
y_pred = (y_probs > 0.5).astype("int32")
accuracy=accuracy_score(y_test, y_pred)
c_matrix=confusion_matrix(y_test, y_pred)
print("Accuracy:", accuracy)
print(c_matrix)

3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
Accuracy: 0.9651162790697675
[[57  2]
 [ 1 26]]


In [28]:
import optuna.visualization as vis

In [29]:
vis.plot_optimization_history(study)

In [30]:
vis.plot_param_importances(study)

In [31]:
vis.plot_parallel_coordinate(study)

## *Data Understanding & Preprocessing*

Load the dataset and explain:

1) Number of features

2) Target variable

3) Data type of each feature

## *Handle* :

1) Missing values (if any)

2) Feature scaling (StandardScaler / MinMaxScaler)

3) Split data into:

4) Train (70%)

5) Validation (15%)

6) Test (15%)

## *Build an ANN Model*

Build a baseline ANN using Keras Sequential API:

1) Input layer

2) At least 2 hidden layers

3) Output layer appropriate to task

4) Compile the model with: One optimizer,One loss function

## *Train and report* :

1) Training loss

2) Validation loss

## *Hyperparameter Tuning using Optuna*

1) Tune at least 4 hyperparameters, such as:

2) Number of hidden layers

3) Activation function

4) Optimizer, Learning rate, Batch size

## *Requirements*:

1) Use Optuna

2) Run at least 20 trials

3) Save best parameters

4) Retrain final model using best configuration

## *Evaluation & Visualization*

Plot:

1) Training vs Validation Loss

2) Evaluate on test set:

3) Regression: MAE, RMSE

4) Classification: Accuracy, Confusion Matrix

## *Answer these questions based on the above experiments*

1) Which hyperparameter had the highest impact?

2) Did tuning always improve performance?

3) One limitation of the Sequential API

## *Submission*
After completing the assignment, please share the GitHub repository link in Discord. Additionally, upload the corresponding commit ID and the GitHub link to the LMS.